
# Construção de um Classificador Binário baseado no algoritmo KNN


## Dependências necessárias

In [955]:
# In[50]:
from __future__ import absolute_import, division, print_function
import pandas as pd
import nltk  
import numpy as np  
nltk.download('punkt')
nltk.download('stopwords')
from nltk.stem.snowball import SnowballStemmer
import heapq
import re  
import io
import math
import csv
# Helper libraries
import matplotlib.pyplot as plt
import seaborn as sns
import functools
import operator
import PIL
import tqdm
import tqdm.auto
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.preprocessing import StandardScaler  
from IPython.display import display
import os

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Carregando variáveis de ambiente

In [957]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# ENV_DATASET_FILENAME = os.environ['INPUT_DATASET_FILENAME']
# ENV_ORDER_DEGREE = os.environ['ORDER_DEGREE']
# ENV_TRAIN_TEST_PROPORTION = os.environ['TRAIN_TEST_PROPORTION']

ENV_DATASET_FILENAME = "locations_to_be_labeled.csv"
ENV_ORDER_DEGREE = 2845
ENV_TRAIN_TEST_PROPORTION = 0.8

nameOfTheFile = ENV_DATASET_FILENAME
print("ENV_DATASET_FILENAME: Nome do arquivo que contém o dataset: " + nameOfTheFile)
print("ORDER_DEGREE: As " + str(ENV_ORDER_DEGREE) + " palavras mais frequentes da bag of words serão consideradas na construção do modelo.")
print("ENV_TRAIN_TEST_PROPORTION: proporção da divisão do dataset em treino e teste: " + str(ENV_TRAIN_TEST_PROPORTION*100) + "%")

ENV_DATASET_FILENAME: Nome do arquivo que contém o dataset: locations_to_be_labeled.csv
ORDER_DEGREE: As 2845 palavras mais frequentes da bag of words serão consideradas na construção do modelo.
ENV_TRAIN_TEST_PROPORTION: proporção da divisão do dataset em treino e teste: 80.0%


## Preparação do conjunto de dados

### Carregando o Dataset

In [958]:
# Lendo as features
questions = pd.read_csv("../" + nameOfTheFile, header=0, usecols=[0])
listOfQuestions = []
for row in questions.values:
    listOfQuestions.append(list(row)[0])

print("Quantidade de perguntas: " + str(len(listOfQuestions)) + "\n")
print("Tweet:\n")
for pergunta in listOfQuestions[0:5]:
    print("- " + pergunta)
print("...")
print("\n")

# Lendo as labels
answers = pd.read_csv("../" + nameOfTheFile, header=0, usecols=[1])
listOfAnswers = []
for answer in answers.values:
    listOfAnswers.append(list(answer)[0])
print("Respostas:\n")
for label in listOfAnswers[0:5]:
    print("- " + str(label))
print("...")

Quantidade de perguntas: 1500

Tweet:

- com o cara faixapreta que venceu a dengue ducatambasco haha foi muito bom te rever brother
- dengue me pegou dnv
- prefeitura de santa cruz divulga plano de ação de enfrentamento a dengue chicungunya e zika
- seguimos na luta contra a dengue chikungunya e o zika vírus com essas três doenças não dá para
- g1 pr segunda morte por dengue em paranaguá é confirmada pela saúde  g1 
...


Respostas:

- 1
- 1
- 0
- 0
- 1
...


### Transformando as labels

Essa transformação vai trocar as labels de número para texto, com o objetivo de facilitar a compreensão e leitura de resultados.
0 - generico
1 - doente

In [959]:
label_palavras = []
for item in listOfAnswers:
    if item==0:
        label_palavras.append("generico")
    else:
        label_palavras.append("doente")
print(label_palavras[0:5])
listOfAnswers = label_palavras

['doente', 'doente', 'generico', 'generico', 'doente']


### Remoção de espaçamentos extras, caracteres especiais e pontuações

In [960]:
for i in range(len(listOfQuestions)):
    listOfQuestions [i] = listOfQuestions [i].lower()
    listOfQuestions [i] = re.sub(r'\W',' ',listOfQuestions [i])
    listOfQuestions [i] = re.sub(r'\s+',' ',listOfQuestions [i])

for question in listOfQuestions[0:5]:
    print("- " + question)
print("...")

- com o cara faixapreta que venceu a dengue ducatambasco haha foi muito bom te rever brother
- dengue me pegou dnv
- prefeitura de santa cruz divulga plano de ação de enfrentamento a dengue chicungunya e zika
- seguimos na luta contra a dengue chikungunya e o zika vírus com essas três doenças não dá para
- g1 pr segunda morte por dengue em paranaguá é confirmada pela saúde g1 
...


### Tokenização dos tweets

In [961]:
def tokenize_sentences(listOfSentences):
    listOfSentencesInTokens = []
    for i in range(len(listOfSentences)):
        tokens = nltk.word_tokenize(listOfSentences[i])
        listOfSentencesInTokens.append(tokens)
    return listOfSentencesInTokens


listOfTokenizedQuestions = tokenize_sentences(listOfQuestions)
for tokenizedItem in listOfTokenizedQuestions[0:5]:
      print("- " + str(tokenizedItem))
print("...")

- ['com', 'o', 'cara', 'faixapreta', 'que', 'venceu', 'a', 'dengue', 'ducatambasco', 'haha', 'foi', 'muito', 'bom', 'te', 'rever', 'brother']
- ['dengue', 'me', 'pegou', 'dnv']
- ['prefeitura', 'de', 'santa', 'cruz', 'divulga', 'plano', 'de', 'ação', 'de', 'enfrentamento', 'a', 'dengue', 'chicungunya', 'e', 'zika']
- ['seguimos', 'na', 'luta', 'contra', 'a', 'dengue', 'chikungunya', 'e', 'o', 'zika', 'vírus', 'com', 'essas', 'três', 'doenças', 'não', 'dá', 'para']
- ['g1', 'pr', 'segunda', 'morte', 'por', 'dengue', 'em', 'paranaguá', 'é', 'confirmada', 'pela', 'saúde', 'g1']
...


### Remoção das stop-words

In [962]:
stopwords = nltk.corpus.stopwords.words('portuguese')
stopwords[:10]
print("Tamanho das stop-words: " + str(len(stopwords)))

def remove_stop_words(listOfTokenizedSentences):
    sentencesWithNoStopWords = []
    for i in range(len(listOfTokenizedSentences)):
        sentenceWithNoStopWord = []
        for j in range(len(listOfTokenizedSentences[i])):
            if(listOfTokenizedSentences[i][j] not in stopwords):
                sentenceWithNoStopWord.append(listOfTokenizedSentences[i][j])
        sentencesWithNoStopWords.append(sentenceWithNoStopWord)
    return sentencesWithNoStopWords

listOfQuestionsWithNoStopWords = remove_stop_words(listOfTokenizedQuestions)
print("Tamanho: " + str(len(listOfQuestionsWithNoStopWords)))

for itemWithNoStopWord in listOfQuestionsWithNoStopWords[0:5]:
    print("- " + str(itemWithNoStopWord))
print("...")

Tamanho das stop-words: 204
Tamanho: 1500
- ['cara', 'faixapreta', 'venceu', 'dengue', 'ducatambasco', 'haha', 'bom', 'rever', 'brother']
- ['dengue', 'pegou', 'dnv']
- ['prefeitura', 'santa', 'cruz', 'divulga', 'plano', 'ação', 'enfrentamento', 'dengue', 'chicungunya', 'zika']
- ['seguimos', 'luta', 'contra', 'dengue', 'chikungunya', 'zika', 'vírus', 'três', 'doenças', 'dá']
- ['g1', 'pr', 'segunda', 'morte', 'dengue', 'paranaguá', 'confirmada', 'saúde', 'g1']
...


### Lematização das tokens

In [963]:
stemmer = SnowballStemmer("portuguese")
print()

def lematizar_tokens(listOfTokenizedSentences):
    stemmed_sentences = []
    for tokenizedSentence in listOfTokenizedSentences:
        stemmed_sentence = []
        for token in tokenizedSentence:
            stemmed_sentence.append(stemmer.stem(token))
        stemmed_sentences.append(stemmed_sentence)
    return stemmed_sentences

def remove_redundancies(stemmed_sentences, labels):
    filtered_list = []
    filtered_labels = []
    for index in range(len(stemmed_sentences)):
        isRedundant = False
        for filtered_sentence in filtered_list:
            if(filtered_sentence == stemmed_sentences[index]):
                isRedundant = True
        if(not isRedundant):
            filtered_list.append(stemmed_sentences[index])
            filtered_labels.append(labels[index])
    return {"features": filtered_list, "labels": filtered_labels}

sentencas_lematizadas = lematizar_tokens(listOfQuestionsWithNoStopWords)
filtered_dataset = remove_redundancies(sentencas_lematizadas, listOfAnswers)

sentencas_lematizadas_e_filtradas = filtered_dataset["features"]
labels_lematizadas_e_filtradas = filtered_dataset["labels"]

# print("----------------------------------------------")
# print("Sentenças lematizadas: ")
# print(sentencas_lematizadas)
# print("Tamanho: " + str(len(sentencas_lematizadas)))
# print("\n")

print("Sentenças lematizadas e filtradas: ")
print("Tamanho: " + str(len(sentencas_lematizadas_e_filtradas)))
for stemmedItem in sentencas_lematizadas_e_filtradas[0:5]:
    print("- " + str(stemmedItem))
print("...")
print("\n")

print("Labels lematizadas e filtradas: ")
print("Tamanho: " + str(len(labels_lematizadas_e_filtradas)))
for stemmedLabel in labels_lematizadas_e_filtradas[0:5]:
    print(stemmedLabel)
print("...")
print("\n")


Sentenças lematizadas e filtradas: 
Tamanho: 1442
- ['car', 'faixapret', 'venc', 'deng', 'ducatambasc', 'hah', 'bom', 'rev', 'broth']
- ['deng', 'peg', 'dnv']
- ['prefeitur', 'sant', 'cruz', 'divulg', 'plan', 'açã', 'enfrent', 'deng', 'chicunguny', 'zik']
- ['segu', 'lut', 'contr', 'deng', 'chikunguny', 'zik', 'vírus', 'três', 'doenc', 'dá']
- ['g1', 'pr', 'segund', 'mort', 'deng', 'paranagu', 'confirm', 'saúd', 'g1']
...


Labels lematizadas e filtradas: 
Tamanho: 1442
doente
doente
generico
generico
doente
...




### Construção da bag-of-words

In [964]:
def create_bag_of_words(listOfTokenizedSentences):
    wordfreq = {}
    for i in range(len(listOfTokenizedSentences)):  #Para cada sentença tokenizada
        for token in listOfTokenizedSentences[i]:       # Para cada token em uma sentença tokenizada
            if token not in wordfreq.keys():
                wordfreq[token] = 1
            else:
                wordfreq[token] += 1
    return wordfreq

wordfreq = create_bag_of_words(sentencas_lematizadas_e_filtradas)
# print(wordfreq)
bag_of_words_size = len(wordfreq.keys())
print("Tamanho da bag of words: " + str(bag_of_words_size))


Tamanho da bag of words: 3246


### Cálculo do parâmetro IDF

In [945]:
order_degree = ENV_ORDER_DEGREE
def get_IDF_wordDictionary(sentencas_lematizadas_e_filtradas, most_freq):
    word_idf_values = {}
    for token in most_freq:
        sentences_containing_word = 0
        for tokenized_sentence in sentencas_lematizadas_e_filtradas:
            if token in tokenized_sentence:
                sentences_containing_word += 1
        word_idf_values[token] = np.log(len(sentencas_lematizadas_e_filtradas)/(1 + sentences_containing_word))
    return word_idf_values

most_freq_dictionary = heapq.nlargest(order_degree , wordfreq, key=wordfreq.get)
print("Palavras mais frequentes e seus respectivos IDFs: ")
idf_wordDictionary = get_IDF_wordDictionary(sentencas_lematizadas_e_filtradas, most_freq_dictionary)
for key in list(idf_wordDictionary.keys())[0:5]:
    print(key + ": " + str(idf_wordDictionary[key]))
print("...")

Palavras mais frequentes e seus respectivos IDFs: 
deng: 0.024571260730505327
mosquit: 2.026762245684408
to: 2.4139739134832223
q: 2.629395418703522
zik: 2.469765273111638
...


### Cálculo do parâmetro TF

In [946]:

def get_TF_wordDictionary(sentencas_lematizadas_e_filtradas,most_freq):
    word_tf_values = {}
    for token in most_freq:
        sent_tf_vector = []
        for tokenized_sentence in sentencas_lematizadas_e_filtradas:
            doc_freq = 0
            for word in tokenized_sentence:
                if token == word:
                    doc_freq += 1
            word_tf = doc_freq/len(tokenized_sentence)
            sent_tf_vector.append(word_tf)
        word_tf_values[token] = sent_tf_vector
    return word_tf_values

tf_wordDictionary = get_TF_wordDictionary(sentencas_lematizadas_e_filtradas, most_freq_dictionary)
# print("Dicionário com os valores TF de cada palavra do corpus em cada sentença do dataset")
# for key in list(tf_wordDictionary.keys())[0:5]:
#     print(key + ": " + str(tf_wordDictionary[key]))
# print("...")

# print("Key / Length")
# for key in tf_wordDictionary:
#     print(key + " / " + str(len(tf_wordDictionary)))

### Build TF-IDF word vectors

In [947]:
def  get_TF_IDF_matrix(word_idf_values, word_tf_values):
    tfidf_values = []
    for token in word_tf_values.keys():
        tfidf_sentences = []
        for tf_sentence in word_tf_values[token]:
            tf_idf_score = tf_sentence * word_idf_values[token]
            tfidf_sentences.append(tf_idf_score)
        tfidf_values.append(tfidf_sentences)
    tf_idf_model = np.asarray(tfidf_values)
    tf_idf_model = np.transpose(tf_idf_model)
    return tf_idf_model

tf_model = get_TF_IDF_matrix(idf_wordDictionary, tf_wordDictionary)
# print(tf_model)
print("Matriz com os vetores de palavras baseados no modelo TF-IDF")
print(tf_model.shape)
# print(tf_model[0:2])

Matriz com os vetores de palavras baseados no modelo TF-IDF
(1442, 2845)


### Divisão do dataset nos conjuntos de treino e teste

In [948]:
treino_teste_proportion = ENV_TRAIN_TEST_PROPORTION

def count_classes(labels):
    classes = []
    for index in range(len(labels)):
        if(labels[index] not in classes):    
            classes.append(labels[index])
    return len(classes)

def segment_classes(features, labels):
    dict = {}
    for index in range(len(labels)):
        if(labels[index] in dict):
            dict[labels[index]].append(features[index])
        else:
            dict[labels[index]] = [features[index]]
    return dict

def split_dataset(features, labels, train_proportion):
#     print("-----------------------------------------------")
#     print("Log: Function split_dataset:")
#     print("-----------------------------------------------")

    dict_classes = segment_classes(features, labels)
#     print("dict_classes:\n" + "number of keys: " + str(len(dict_classes.keys())))
#     for key in dict_classes:
#         print("   key: " + str(key) + "\n" + "   value: " +  str(len(dict_classes[key])) + " elements in array \n")

    train_features = []
    train_labels = []
    test_features = []
    test_labels = []

    for key_class in dict_classes:

        remove_from_class = math.ceil(len(dict_classes[key_class]) * train_proportion)
#         print("remove_from_class: " + str(key_class) + ", " + str(remove_from_class) + " elements go to train set")

        for index in range(len(dict_classes[key_class])):
            if(index <= remove_from_class -1):
                train_features.append(dict_classes[key_class][index])
                train_labels.append(key_class)
            else:
                test_features.append(dict_classes[key_class][index])
                test_labels.append(key_class)

#     print("\n")
#     print("train_features: " + str(len(train_features)))  
#     print("train_labels: " + str(len(train_labels)))
#     print("test_features: " + str(len(test_features)))
#     print("test_labels: " + str(len(test_labels)))        
#     print("-----------------------------------------------")
#     print("\n")
    return { "train":{"features": np.array(train_features), "labels": np.array(train_labels)}, "test": {"features": np.array(test_features), "labels": np.array(test_labels)}}

# sentence_vectors é uma matriz que contem as sentenças (sem stopwords, lematizadas e filtradas) vetorizadas. Dimensão atual: (81x57)
# listOfAnswers corresponde às labels (respostas) correspondentes à cada sentença(pergunta) em sentence_vectors. tamanho atual: (105)
# datasets = split_dataset(sentence_vectors, labels_lematizadas_e_filtradas, 0.7)
datasets = split_dataset(tf_model, labels_lematizadas_e_filtradas, treino_teste_proportion)

dataset_train = datasets["train"] # É um dicionário com o formato {"features: np.array(), "labels": np.array()}
dataset_test = datasets["test"] # É um dicionário com o formato {"features: np.array(), "labels": np.array()}

#datasets["test"] = np.array(datasets["test"])

print("Conjunto de treino:")
print("Tamanho: " + str(len(dataset_train["features"])))
# print(dataset_train["features"])
# for feat in dataset_train["features"]:
#   print(feat)
# for label in dataset_train["labels"]:
#   print(label)
print("\n")
print("Conjunto de teste:")
print("Tamanho: " + str(len(dataset_test["features"])))
# print(dataset_test["features"])
# print(dataset_test["labels"])

Conjunto de treino:
Tamanho: 1155


Conjunto de teste:
Tamanho: 287


### Codificação das labels com one-hot-encoding

In [949]:
def dense_to_one_hot(y_train, num_classes):
    y_one_hot = []
    encode_output = encode(y_train)
    labels_encoded = encode_output["labels"]
    encoding_dict = encode_output["encoding_dict"]
    for index in range(0,len(labels_encoded)): 
        nova_label = [0]*num_classes
        nova_label[int(labels_encoded[index])]=1
        y_one_hot.append(nova_label)
    return [np.array(y_one_hot),encoding_dict]

# labels é o array com as labels, isto é, os valores indicando se tweets são de pessoas doentes ou não
# 1 - pessoa doente
# 0 - tweet genérico
# A função encode verifica quantas classes diferentes à no array labels. No caso temos 2 classes, 0 e 1.
# Para cada uma das classes encontradas, é atribuído um número em ordem crescente.
# Para a classe 1 (tweets de pessoas doentes), está sendo atribuído o valor 0
# Para a classe 0 (tweets genéricos), está sendo atribuído o valor 1
# O valor atribuído a cada classe representa o index da posição no vetor one_hot_encoded que será marcado como 1 para representar a classe
# na representação one_hot_encoded.
# Isso significa que:
# - um tweet de pessoa doente vai receber uma label com a seguinte codificação: [1 0]
# - um tweet genérico vai receber uma label com a seguinte codificação: [0 1]
def encode(labels):
    dict_classes = {}
    count = 0
    labels_encoded = []
    for label in labels:
        if label not in dict_classes:
            dict_classes[label] = count
            count += 1
    for label in labels:
        labels_encoded.append(dict_classes[label])
#     print("Dicionário de codificação one_hot, formato: {classe:index}: " + str(dict_classes))
    return {"labels":labels_encoded, "encoding_dict":dict_classes}

def is_encoding_uniform(*dicts):
    is_equal = True
    dict_ref = dicts[0]
    for index in range(1, len(dicts)):
        if dicts[index] != dict_ref:
            is_equal = False
            return is_equal
    return is_equal


n_classes = count_classes(dataset_test["labels"])
[train_labels_one_hot_encoded,encoding_dict1] = dense_to_one_hot(dataset_train["labels"], n_classes)
[test_labels_one_hot_encoded,encoding_dict2] = dense_to_one_hot(dataset_test["labels"], n_classes)


encoding_dictionary = {}
# O código abaixo verifica se os conjuntos de labels passaram pela mesma codificação one_hot
if not is_encoding_uniform(encoding_dict1, encoding_dict2):
    print("ATENÇÃO! Há um problema com a codificação one_hot dos conjuntos de labels. Há conjuntos que receberam codificações diferentes!")
else:
    print("Ok, a codificação foi a mesma para todos os conjuntos de label. Continuando...")
    print("\n")
    encoding_dictionary = encoding_dict1

print("Labels de treino codificadas:")
print("Tamanho: " + str(train_labels_one_hot_encoded.shape))
print(str(train_labels_one_hot_encoded[0:5]))
print('...')
print("\n")

print("Labels de teste codificadas: ")
print("Tamanho: " + str(test_labels_one_hot_encoded.shape))
print(str(test_labels_one_hot_encoded[0:5]))
print('...')

Ok, a codificação foi a mesma para todos os conjuntos de label. Continuando...


Labels de treino codificadas:
Tamanho: (1155, 2)
[[1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]]
...


Labels de teste codificadas: 
Tamanho: (287, 2)
[[1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]]
...


## Construção do modelo

In [950]:
classifier = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=48, p=2,
           weights='distance')


In [951]:
scaler = StandardScaler()  
scaler.fit(dataset_train["features"])

# dataset_train_escalonado = scaler.transform(dataset_train["features"])  
# dataset_test_escalonado = scaler.transform(dataset_test["features"]) 

dataset_train_escalonado = dataset_train["features"] 
dataset_test_escalonado = dataset_test["features"]

## Treinamento do modelo

In [952]:
classifier.fit(dataset_train_escalonado, train_labels_one_hot_encoded) 

KNeighborsClassifier(n_neighbors=48, weights='distance')

## Fazendo predições sobre o conjunto de teste

In [953]:
y_pred = classifier.predict(dataset_test_escalonado)
print(y_pred[0:5])
print("...")

[[0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]]
...


## Validando o modelo

In [954]:
def from_onehot_to_encoded(labels_onehot):
    labels_encoded = []
    for element in labels_onehot:
        for index in range(len(element)):
            if(element[index]==1):
                labels_encoded.append(index)
    return labels_encoded

def matConfusion(y_test, y_pred):
    matConf = confusion_matrix(y_test, y_pred)
    acc = np.trace(matConf)/np.sum(matConf)
    terr  = 1 - acc
    return matConf,terr,acc

def convert_to_label(encoded_list,encoding_dict):
    labels_decoded = []
    for encoded_item in encoded_list:
        label = get_key(encoded_item, encoding_dict)
        labels_decoded.append(label)
    return labels_decoded

def get_key(val, my_dict): 
    for key, value in my_dict.items(): 
            if val == value: 
                return key 
    return "Key does not exist"

def validate(NUM_LABELS, matConf):
    acc_classes = np.zeros(NUM_LABELS)
    for index in range(NUM_LABELS):
        acc_classes[index] = matConf[index,index]/sum(matConf[index,:])
    acc_list = []
    for acc in acc_classes:
        acc_list.append(acc)
    return acc_list

y_pred_validador = np.argmax(y_pred,1)
# print(y_pred_validador[0:5])

test_labels_encoded = from_onehot_to_encoded(test_labels_one_hot_encoded)
matConf,terr,acc = matConfusion(test_labels_encoded, y_pred_validador)

print("oder_degree: " + str(order_degree))
#Plotando a matriz de confusão
print('Matrix confusion')
print(matConf)
print('err = %f%s' %(100*terr,'%'))
print('acuracy = %f%s' %(100*acc,'%'))

# Acurácia para cada classe
NUM_LABELS = 2

val_data = validate(NUM_LABELS, matConf)

c=0
for acc in val_data:
    print("Accuracy para classe " + str(c) + ": " + str(acc*100) + ".")
    c += 1

oder_degree: 2845
Matrix confusion
[[ 68  22]
 [ 50 147]]
err = 25.087108%
acuracy = 74.912892%
Accuracy para classe 0: 75.55555555555556.
Accuracy para classe 1: 74.61928934010153.


## Rotina de treinamento

Para usar essa rotina, rode o seu notebook até construir a bag-of-words.

In [ ]:
take_from_bag = bag_of_words_size-1
escalonar_features = True
weights_param = "distance"

treino_teste_proportion = 0.8

take_bag_limit = 150
k_neighbor_start_point = 1
k_neighbor_limit = 200

header = ["train_test_proportion", 
          "weights_param", 
          "escalonar_features",
          "order_degree", 
          "k_neighbor", 
          "error", 
          "acc", 
          "acc_classe0", 
          "acc_classe1"
         ]

with open("./" + "training_report1.csv", "w") as reportFile:
    
    writer = csv.writer(reportFile, delimiter=",")
    
    # Escreve header do relatório
    writer.writerow(header)
    
    while(take_from_bag > take_bag_limit):  

        most_freq_dictionary = heapq.nlargest(take_from_bag , wordfreq, key=wordfreq.get)
        # IDF
        idf_wordDictionary = get_IDF_wordDictionary(sentencas_lematizadas_e_filtradas, most_freq_dictionary)
        # TF
        tf_wordDictionary = get_TF_wordDictionary(sentencas_lematizadas_e_filtradas, most_freq_dictionary)
        # TF-IDF
        tf_model = get_TF_IDF_matrix(idf_wordDictionary, tf_wordDictionary)

        # Split dataset
        datasets = split_dataset(tf_model, labels_lematizadas_e_filtradas, treino_teste_proportion)
        dataset_train = datasets["train"] # É um dicionário com o formato {"features: np.array(), "labels": np.array()}
        dataset_test = datasets["test"] # É um dicionário com o formato {"features: np.array(), "labels": np.array()}

        # One-hot-encoding
        n_classes = count_classes(dataset_test["labels"])
        [train_labels_one_hot_encoded,encoding_dict1] = dense_to_one_hot(dataset_train["labels"], n_classes)
        [test_labels_one_hot_encoded,encoding_dict2] = dense_to_one_hot(dataset_test["labels"], n_classes)
        encoding_dictionary = {}
        # O código abaixo verifica se os conjuntos de labels passaram pela mesma codificação one_hot
        if not is_encoding_uniform(encoding_dict1, encoding_dict2):
            print("ATENÇÃO! Há um problema com a codificação one_hot dos conjuntos de labels. Há conjuntos que receberam codificações diferentes!")
        else:
            encoding_dictionary = encoding_dict1

        k_neighbor = k_neighbor_start_point
        while(k_neighbor< k_neighbor_limit):

            # Build model
            classifier = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
            metric_params=None, n_jobs=None, n_neighbors=k_neighbor, p=2,
            weights=weights_param)

            # Escalonar?
            dataset_train_escalonado = []
            dataset_test_escalonado = []
            if escalonar_features:
                print("Escalonando...")
                scaler = StandardScaler()  
                scaler.fit(dataset_train["features"])
                dataset_train_escalonado = scaler.transform(dataset_train["features"])  
                dataset_test_escalonado = scaler.transform(dataset_test["features"])
            else:
                dataset_train_escalonado = dataset_train["features"] 
                dataset_test_escalonado = dataset_test["features"]

            # Treino
            classifier.fit(dataset_train_escalonado, train_labels_one_hot_encoded) 

            # Teste
            y_pred = classifier.predict(dataset_test_escalonado)

            # Validação
            y_pred_validador = np.argmax(y_pred,1)
            test_labels_encoded = from_onehot_to_encoded(test_labels_one_hot_encoded)
            matConf,terr,general_acc = matConfusion(test_labels_encoded, y_pred_validador)
            NUM_LABELS = 2
            val_data = validate(NUM_LABELS, matConf)
            c=0
            acc_classes = []
            for acc in val_data:
                acc_classes.append(str(acc))
                c += 1
           
            print("\n")
            print("bag:" + str(take_from_bag) + "/" + str(take_bag_limit))
            print("k:" + str(k_neighbor) + "/" + str(k_neighbor_limit))
            
            writer.writerow(
                [treino_teste_proportion, 
                   weights_param, 
                   escalonar_features, 
                   take_from_bag, 
                   k_neighbor, 
                   terr, 
                   general_acc, 
                   acc_classes[0], 
                   acc_classes[1]]
                 )

            k_neighbor += 1
        take_from_bag -= 25

Escalonando...


bag:3245/150
k:1/200
Escalonando...


bag:3245/150
k:2/200
Escalonando...


bag:3245/150
k:3/200
Escalonando...


bag:3245/150
k:4/200
Escalonando...


bag:3245/150
k:5/200
Escalonando...


bag:3245/150
k:6/200
Escalonando...


bag:3245/150
k:7/200
Escalonando...


bag:3245/150
k:8/200
Escalonando...


bag:3245/150
k:9/200
Escalonando...


bag:3245/150
k:10/200
Escalonando...


bag:3245/150
k:11/200
Escalonando...


bag:3245/150
k:12/200
Escalonando...


bag:3245/150
k:13/200
Escalonando...


bag:3245/150
k:14/200
Escalonando...


bag:3245/150
k:15/200
Escalonando...


bag:3245/150
k:16/200
Escalonando...


bag:3245/150
k:17/200
Escalonando...


bag:3245/150
k:18/200
Escalonando...


bag:3245/150
k:19/200
Escalonando...


bag:3245/150
k:20/200
Escalonando...


bag:3245/150
k:21/200
Escalonando...


bag:3245/150
k:22/200
Escalonando...


bag:3245/150
k:23/200
Escalonando...


bag:3245/150
k:24/200
Escalonando...


bag:3245/150
k:25/200
Escalonando...


bag:3245/150
k:26



bag:3220/150
k:10/200
Escalonando...


bag:3220/150
k:11/200
Escalonando...


bag:3220/150
k:12/200
Escalonando...


bag:3220/150
k:13/200
Escalonando...


bag:3220/150
k:14/200
Escalonando...


bag:3220/150
k:15/200
Escalonando...


bag:3220/150
k:16/200
Escalonando...


bag:3220/150
k:17/200
Escalonando...


bag:3220/150
k:18/200
Escalonando...


bag:3220/150
k:19/200
Escalonando...


bag:3220/150
k:20/200
Escalonando...


bag:3220/150
k:21/200
Escalonando...


bag:3220/150
k:22/200
Escalonando...


bag:3220/150
k:23/200
Escalonando...


bag:3220/150
k:24/200
Escalonando...


bag:3220/150
k:25/200
Escalonando...


bag:3220/150
k:26/200
Escalonando...


bag:3220/150
k:27/200
Escalonando...


bag:3220/150
k:28/200
Escalonando...


bag:3220/150
k:29/200
Escalonando...


bag:3220/150
k:30/200
Escalonando...


bag:3220/150
k:31/200
Escalonando...


bag:3220/150
k:32/200
Escalonando...


bag:3220/150
k:33/200
Escalonando...


bag:3220/150
k:34/200
Escalonando...


bag:3220/150
k:35/200
E



bag:3195/150
k:19/200
Escalonando...


bag:3195/150
k:20/200
Escalonando...


bag:3195/150
k:21/200
Escalonando...


bag:3195/150
k:22/200
Escalonando...


bag:3195/150
k:23/200
Escalonando...


bag:3195/150
k:24/200
Escalonando...


bag:3195/150
k:25/200
Escalonando...


bag:3195/150
k:26/200
Escalonando...


bag:3195/150
k:27/200
Escalonando...


bag:3195/150
k:28/200
Escalonando...


bag:3195/150
k:29/200
Escalonando...


bag:3195/150
k:30/200
Escalonando...


bag:3195/150
k:31/200
Escalonando...


bag:3195/150
k:32/200
Escalonando...


bag:3195/150
k:33/200
Escalonando...


bag:3195/150
k:34/200
Escalonando...


bag:3195/150
k:35/200
Escalonando...


bag:3195/150
k:36/200
Escalonando...


bag:3195/150
k:37/200
Escalonando...


bag:3195/150
k:38/200
Escalonando...


bag:3195/150
k:39/200
Escalonando...


bag:3195/150
k:40/200
Escalonando...


bag:3195/150
k:41/200
Escalonando...


bag:3195/150
k:42/200
Escalonando...


bag:3195/150
k:43/200
Escalonando...


bag:3195/150
k:44/200
E



bag:3170/150
k:28/200
Escalonando...


bag:3170/150
k:29/200
Escalonando...


bag:3170/150
k:30/200
Escalonando...


bag:3170/150
k:31/200
Escalonando...


bag:3170/150
k:32/200
Escalonando...


bag:3170/150
k:33/200
Escalonando...


bag:3170/150
k:34/200
Escalonando...


bag:3170/150
k:35/200
Escalonando...


bag:3170/150
k:36/200
Escalonando...


bag:3170/150
k:37/200
Escalonando...


bag:3170/150
k:38/200
Escalonando...


bag:3170/150
k:39/200
Escalonando...


bag:3170/150
k:40/200
Escalonando...


bag:3170/150
k:41/200
Escalonando...


bag:3170/150
k:42/200
Escalonando...


bag:3170/150
k:43/200
Escalonando...


bag:3170/150
k:44/200
Escalonando...


bag:3170/150
k:45/200
Escalonando...


bag:3170/150
k:46/200
Escalonando...


bag:3170/150
k:47/200
Escalonando...


bag:3170/150
k:48/200
Escalonando...


bag:3170/150
k:49/200
Escalonando...


bag:3170/150
k:50/200
Escalonando...


bag:3170/150
k:51/200
Escalonando...


bag:3170/150
k:52/200
Escalonando...


bag:3170/150
k:53/200
E



bag:3145/150
k:37/200
Escalonando...


bag:3145/150
k:38/200
Escalonando...


bag:3145/150
k:39/200
Escalonando...


bag:3145/150
k:40/200
Escalonando...


bag:3145/150
k:41/200
Escalonando...


bag:3145/150
k:42/200
Escalonando...


bag:3145/150
k:43/200
Escalonando...


bag:3145/150
k:44/200
Escalonando...


bag:3145/150
k:45/200
Escalonando...


bag:3145/150
k:46/200
Escalonando...


bag:3145/150
k:47/200
Escalonando...


bag:3145/150
k:48/200
Escalonando...


bag:3145/150
k:49/200
Escalonando...


bag:3145/150
k:50/200
Escalonando...


bag:3145/150
k:51/200
Escalonando...


bag:3145/150
k:52/200
Escalonando...


bag:3145/150
k:53/200
Escalonando...


bag:3145/150
k:54/200
Escalonando...


bag:3145/150
k:55/200
Escalonando...


bag:3145/150
k:56/200
Escalonando...


bag:3145/150
k:57/200
Escalonando...


bag:3145/150
k:58/200
Escalonando...


bag:3145/150
k:59/200
Escalonando...


bag:3145/150
k:60/200
Escalonando...


bag:3145/150
k:61/200
Escalonando...


bag:3145/150
k:62/200
E

...



bag:1720/150
k:134/200
Escalonando...


bag:1720/150
k:135/200
Escalonando...


bag:1720/150
k:136/200
Escalonando...


bag:1720/150
k:137/200
Escalonando...


bag:1720/150
k:138/200
Escalonando...


bag:1720/150
k:139/200
Escalonando...


bag:1720/150
k:140/200
Escalonando...


bag:1720/150
k:141/200
Escalonando...


bag:1720/150
k:142/200
Escalonando...


bag:1720/150
k:143/200
Escalonando...


bag:1720/150
k:144/200
Escalonando...


bag:1720/150
k:145/200
Escalonando...


bag:1720/150
k:146/200
Escalonando...


bag:1720/150
k:147/200
Escalonando...


bag:1720/150
k:148/200
Escalonando...


bag:1720/150
k:149/200
Escalonando...


bag:1720/150
k:150/200
Escalonando...


bag:1720/150
k:151/200
Escalonando...


bag:1720/150
k:152/200
Escalonando...


bag:1720/150
k:153/200
Escalonando...


bag:1720/150
k:154/200
Escalonando...


bag:1720/150
k:155/200
Escalonando...


bag:1720/150
k:156/200
Escalonando...


bag:1720/150
k:157/200
Escalonando...


bag:1720/150
k:158/200
Escalonando...




bag:1695/150
k:143/200
Escalonando...


bag:1695/150
k:144/200
Escalonando...


bag:1695/150
k:145/200
Escalonando...


bag:1695/150
k:146/200
Escalonando...


bag:1695/150
k:147/200
Escalonando...


bag:1695/150
k:148/200
Escalonando...


bag:1695/150
k:149/200
Escalonando...


bag:1695/150
k:150/200
Escalonando...


bag:1695/150
k:151/200
Escalonando...


bag:1695/150
k:152/200
Escalonando...


bag:1695/150
k:153/200
Escalonando...


bag:1695/150
k:154/200
Escalonando...


bag:1695/150
k:155/200
Escalonando...


bag:1695/150
k:156/200
Escalonando...


bag:1695/150
k:157/200
Escalonando...


bag:1695/150
k:158/200
Escalonando...


bag:1695/150
k:159/200
Escalonando...


bag:1695/150
k:160/200
Escalonando...


bag:1695/150
k:161/200
Escalonando...


bag:1695/150
k:162/200
Escalonando...


bag:1695/150
k:163/200
Escalonando...


bag:1695/150
k:164/200
Escalonando...


bag:1695/150
k:165/200
Escalonando...


bag:1695/150
k:166/200
Escalonando...


bag:1695/150
k:167/200
Escalonando...




bag:1670/150
k:152/200
Escalonando...


bag:1670/150
k:153/200
Escalonando...


bag:1670/150
k:154/200
Escalonando...


bag:1670/150
k:155/200
Escalonando...


bag:1670/150
k:156/200
Escalonando...


bag:1670/150
k:157/200
Escalonando...


bag:1670/150
k:158/200
Escalonando...


bag:1670/150
k:159/200
Escalonando...


bag:1670/150
k:160/200
Escalonando...


bag:1670/150
k:161/200
Escalonando...


bag:1670/150
k:162/200
Escalonando...


bag:1670/150
k:163/200
Escalonando...


bag:1670/150
k:164/200
Escalonando...


bag:1670/150
k:165/200
Escalonando...


bag:1670/150
k:166/200
Escalonando...


bag:1670/150
k:167/200
Escalonando...


bag:1670/150
k:168/200
Escalonando...


bag:1670/150
k:169/200
Escalonando...


bag:1670/150
k:170/200
Escalonando...


bag:1670/150
k:171/200
Escalonando...


bag:1670/150
k:172/200
Escalonando...


bag:1670/150
k:173/200
Escalonando...


bag:1670/150
k:174/200
Escalonando...


bag:1670/150
k:175/200
Escalonando...


bag:1670/150
k:176/200
Escalonando...




bag:1645/150
k:161/200
Escalonando...


bag:1645/150
k:162/200
Escalonando...


bag:1645/150
k:163/200
Escalonando...


bag:1645/150
k:164/200
Escalonando...


bag:1645/150
k:165/200
Escalonando...


bag:1645/150
k:166/200
Escalonando...


bag:1645/150
k:167/200
Escalonando...


bag:1645/150
k:168/200
Escalonando...


bag:1645/150
k:169/200
Escalonando...


bag:1645/150
k:170/200
Escalonando...


bag:1645/150
k:171/200
Escalonando...


bag:1645/150
k:172/200
Escalonando...


bag:1645/150
k:173/200
Escalonando...


bag:1645/150
k:174/200
Escalonando...


bag:1645/150
k:175/200
Escalonando...


bag:1645/150
k:176/200
Escalonando...


bag:1645/150
k:177/200
Escalonando...


bag:1645/150
k:178/200
Escalonando...


bag:1645/150
k:179/200
Escalonando...


bag:1645/150
k:180/200
Escalonando...


bag:1645/150
k:181/200
Escalonando...


bag:1645/150
k:182/200
Escalonando...


bag:1645/150
k:183/200
Escalonando...


bag:1645/150
k:184/200
Escalonando...


bag:1645/150
k:185/200
Escalonando...




bag:1620/150
k:170/200
Escalonando...


bag:1620/150
k:171/200
Escalonando...


bag:1620/150
k:172/200
Escalonando...


bag:1620/150
k:173/200
Escalonando...


bag:1620/150
k:174/200
Escalonando...


bag:1620/150
k:175/200
Escalonando...


bag:1620/150
k:176/200
Escalonando...


bag:1620/150
k:177/200
Escalonando...


bag:1620/150
k:178/200
Escalonando...


bag:1620/150
k:179/200
Escalonando...


bag:1620/150
k:180/200
Escalonando...


bag:1620/150
k:181/200
Escalonando...


bag:1620/150
k:182/200
Escalonando...


bag:1620/150
k:183/200
Escalonando...


bag:1620/150
k:184/200
Escalonando...


bag:1620/150
k:185/200
Escalonando...


bag:1620/150
k:186/200
Escalonando...


bag:1620/150
k:187/200
Escalonando...


bag:1620/150
k:188/200
Escalonando...


bag:1620/150
k:189/200
Escalonando...


bag:1620/150
k:190/200
Escalonando...


bag:1620/150
k:191/200
Escalonando...


bag:1620/150
k:192/200
Escalonando...


bag:1620/150
k:193/200
Escalonando...


bag:1620/150
k:194/200
Escalonando...
